In [1]:
import pandas as pd
import numpy as np
from tasks.pipeline import indicator_pipeline
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from index.utils import ISO_to_Everything

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def preprocess():
    df = (
        pd.read_excel('data/indicator/GV2/raw/GV2_IRENA.M.xlsx')
          #.rename(columns={'ISO-code': 'ISO'})
          #.groupby(['ISO', 'Year'])['Amount (2019 USD million)'].sum()
    )
    return df#.to_frame('Value').reset_index()
df = preprocess()

In [35]:
df#.query('ISO == "FRA"')

,ISO-code,Country/Area,Region,Project,Donor,Agency,Year,Category,Technology,Sub-technology,Finance Group,Finance Type,Source,Reference Date,Amount (2019 USD million)
0,DZA,Algeria,Africa,NaN,Japan,Japanese International Co-operation Agency,2019,Renewables,Multiple renewables,Multiple renewables,Grants,Standard grant,2019 OECD Credit Report System (CRS) Database,2019-12-31 00:00:00,0.016066
1,DZA,Algeria,Africa,P209 - SCAC - Algérie - Projet - Crédits d'int...,France,Ministry of Foreign Affairs,2019,Renewables,Solar energy,On-grid Solar photovoltaic,Grants,Standard grant,2019 OECD Credit Report System (CRS) Database,2019-01-01 00:00:00,0.015672
2,DZA,Algeria,Africa,P209 - SCAC - Algérie - Projet - Crédits d'int...,France,Ministry of Foreign Affairs,2019,Renewables,Solar energy,On-grid Solar photovoltaic,Grants,Standard grant,2019 OECD Credit Report System (CRS) Database,2019-01-01 00:00:00,0.015672
3,DZA,Algeria,Africa,P209 - SCAC - Algérie - Projet - Crédits d'int...,France,Ministry of Foreign Affairs,2019,Renewables,Solar energy,On-grid Solar photovoltaic,Grants,Standard grant,2019 OECD Credit Report System (CRS) Database,2019-01-01 00:00:00,0.001791
4,DZA,Algeria,Africa,TC AGGREGATED ACTIVITIES,Japan,Japanese International Co-operation Agency,2013,Renewables,Multiple renewables,Multiple renewables,Grants,Standard grant,2013 OECD Credit Report System (CRS) Database,2013-01-01 00:00:00,0.928327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15922,VEN,Venezuela,South America,Climate Change Workshop,United Kingdom,Foreign & Commonwealth Office,2014,Renewables,Multiple renewables,Multiple renewables,Grants,Standard grant,2014 OECD Credit Report System (CRS) Database,2014-01-01 00:00:00,0.002444
15923,VEN,Venezuela,South America,Rehabilitation of Units 1 to 6 of Powerhouse I...,Inter-American Development Bank,Ordinary Capital,2010,Renewables,Renewable hydropower,Renewable hydropower,Debt,Standard loan,2010 OECD Credit Report System (CRS) Database,2010-10-27 00:00:00,685.681463
15924,VEN,Venezuela,South America,"A Powerhouse 1 Central Simon Bolivar"" Project",Inter-American Development Bank,Inter-American Development Bank,2015,Renewables,Renewable hydropower,Renewable hydropower,Grants,Standard grant,https://www.iadb.org/project/VE-T1050,42275,0.351932
15925,VEN,Venezuela,South America,Support to the Implementation of the Guri Reha...,Inter-American Development Bank,Inter-American Development Bank,2018,Renewables,Renewable hydropower,Renewable hydropower,Grants,Standard grant,https://www.iadb.org/project/VE-T1064,43423,0.392393


# Food security

In [ ]:
indicator_pipeline('GS1')

In [ ]:
data = pd.read_csv('data/indicator/GS1/processed/GS1_SDG.csv')
ST = pd.DataFrame({"Indicator": ['GS1'], "Number of targets": 1, "Relation": 'negative', 'Target 1': 0, 'Target 2': np.nan}).set_index('Indicator')
df = (
    data.groupby('Year')
        .apply(lambda x: GreenGrowthScaler().normalize(x.rename(columns={'Value': 'GS1'}).set_index('ISO')[['GS1']], ST)).reset_index()
        .pivot(index=['ISO'], columns=['Year'], values='GS1')
        .dropna(axis=1, how='all')
        .reset_index()
)
#ISO_to_Everything(df)#.to_csv('data/indicator/GS1/processed/food_insecurity_ts.csv')

In [ ]:
# # This part is added to data/indicator/TMP/preprocess.py

# def process_TMP():
#     df = (
#         pd.read_csv('data/indicator/TMP/raw/TMP_IEA.M.csv')
#           .melt(id_vars=['Country', 'Mode/vehicle type', 'Indicator'], var_name=['Year'], value_name='Value')
#           .rename(columns={'Mode/vehicle type': 'mode'})
#           .assign(Indicator=lambda x: x.Indicator.str.strip())
#           .query("mode == 'Total passenger transport' and Indicator == 'Passenger-kilometres energy intensity (MJ/pkm)'")
#           .drop(columns=['mode', 'Indicator'])
#           .dropna()
#     )
#     return df
    
    
# config_TMP = {'Variable': 'TMP',
#              'function': process_TMP,
#              'Description': 'Total passenger transport Passenger-kilometres energy intensity (MJ/pkm)',
#              'Source': 'IEA',
#              'URL': 'https://www.iea.org/data-and-statistics/data-product/energy-efficiency-indicators'}

In [ ]:
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler

indicator = (pd.read_csv('data/indicator/TMP/processed/TMP_origin.M.csv')
               .query("Year == 2019")
               .set_index("ISO")[['Value']]
                .rename(columns={"Value": 'TMP'})
            ) # Data Frame with indicators here there is just "TMP", must be for a single year ! 

ST = pd.DataFrame({"Indicator": ['TMP'], "Number of targets": 1, "Relation": 'negative', 'Target 1': 1.104, 'Target 2': np.nan}).set_index('Indicator')
# ST stands for sustainable target:
# - Number of targets is almost always 1 so leave it as 1 by default
# - Relation is wether high value is good/bad for the environnment. Here, lower energy intensity is better so the relation is negative
# - Target 1 and 2 are for the value of the target. To compute the target take the average of the 5 best countries

In [ ]:
Normalized_Indicator = GreenGrowthScaler().normalize(indicator, ST) # Call this and that's it.
Normalized_Indicator

In [ ]:
Normalized_Indicator.to_csv('data/indicator/TMP/processed/TMP_Normalized_origin.M.csv') # You can share this

In [ ]:
pd.read_csv('data/sustainable_targets/ST_2020.csv', index_col=0) # Just so you can see what it looks like for other indicators

#  Universal acess

In [ ]:
import pandas as pd
import numpy as np
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from sklearn.preprocessing import MinMaxScaler

def process():
    df_1 = pd.read_csv('data/indicator/GS3/raw/Rural access index.csv').dropna(axis=1).rename(columns={'Value': 'Rural access index'})
    df_2 = pd.read_csv('data/indicator/GS3/raw/Percentage of female workers in transport.csv').dropna(axis=1).rename(columns={'Value': 'Percentage female workers in transport'})
    df_3 = pd.read_csv('data/indicator/GS3/raw/Rapid Transit to Resident Ratio.csv').dropna(axis=1).rename(columns={'Value': 'Rapid Transit to Resident Ratio'})

    
    df = pd.concat([df_1.set_index(['Country']), df_2.set_index(['Country']), df_3.set_index(['Country'])], axis=1).drop(columns=['Year']).dropna()

    
    df_norm = pd.DataFrame(MinMaxScaler(feature_range=(1, 100)).fit_transform(df), columns=df.columns, index=df.index)
    return df_norm.mean(axis=1).to_frame(name='Value').assign(Year=2020).reset_index()

In [ ]:
df = process()
#df = add_ISO(df.reset_index())
#ISO_to_Everything(df)[['Country', 'Universal Acess']].to_csv('data/indicator/GS3/processed/normalized_universal_access_index.csv')

In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from sklearn.preprocessing import MinMaxScaler
from processing.utils import add_ISO
from index.utils import ISO_to_Everything

def process():
    df = pd.read_csv('data/indicator/GS3/raw/Logistics performance index.csv').dropna(axis=1).rename(columns={'Value': 'Logistics performance index'}).set_index('Country')

    
    
    
    ST = pd.DataFrame({"Indicator": ['Logistics performance index'],
                   "Number of targets": [1],
                   "Relation": ['positive'],
                   'Target 1': [4.069],
                   'Target 2': [np.nan]}).set_index('Indicator')

    #df_norm = GreenGrowthScaler().normalize(df[['Logistics performance index']], ST) # Call this and that's it.
    df_norm = pd.DataFrame(MinMaxScaler(feature_range=(1, 100)).fit_transform(df), columns=df.columns, index=df.index)

    return df_norm

In [ ]:
df = process()
df = add_ISO(df['Logistics performance index'].reset_index())

In [ ]:
ISO_to_Everything(df)[['Country', 'Logistics performance index']].to_csv('data/indicator/GS3/processed/normalized_Logistics_performance_index.csv')

In [ ]:
pd.read_csv('data/indicator/GS3/processed/normalized_Logistics_performance_index.csv')

In [ ]:
pd.read_csv('data/indicator/GS3/processed/normalized_universal_access_index.csv')